# **LLM Routing wtih Apigee**

<table align="left">
    <td style="text-align: center">
        <a href="https://colab.research.google.com/github/ssvaidyanathan/apigee-samples/blob/main/llm-routing/llm_routing_v1.ipynb">
          <img src="images/colab-logo-32px.png" alt="Google Colaboratory logo\"><br> Open in Colab
        </a>
      </td>
      <td style="text-align: center">
        <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fssvaidyanathan%2Fapigee-samples%2Fmain%2Fllm-routing%2Fllm_routing_v1.ipynb">
          <img width="32px" src="images/colab-ent.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
        </a>
      </td>    
      <td style="text-align: center">
        <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/ssvaidyanathan/apigee-samples/main/llm-routing/llm_routing_v1.ipynb">
          <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
        </a>
      </td>
      <td style="text-align: center">
        <a href="https://github.com/ssvaidyanathan/apigee-samples/blob/main/llm-routing/llm_routing_v1.ipynb">
          <img src="images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
        </a>
      </td>
</table>
<br />
<br />
<br />

# Routing Sample

- This is a sample Apigee proxy to demonstrate the routing capabilities of Apigee across different LLM providers. In this sample we will use Google VertexAI and Anthropic as the LLM providers
- The framework will easily help onboarding other providers using configurations

![architecture](./images/arch.jpg)

# Benefits of Routing with Apigee:

* **Configuration Driven Routing**: All the routing logic are driven through configuration which makes onboarding very easy
* **Security**: Irrespective of the model and providers, Apigee will secure the endpoints and
* **Consistency**: Apigee can offer that layer of consistency to work with any LLM SDKs that are being used

## Setup

Use the following GCP CloudShell tutorial. Follow the instructions to deploy the sample.

[![Open in Cloud Shell](https://gstatic.com/cloudssh/images/open-btn.png)](https://ssh.cloud.google.com/cloudshell/open?cloudshell_git_repo=https://github.com/ssvaidyanathan/apigee-samples&cloudshell_git_branch=main&cloudshell_workspace=.&cloudshell_tutorial=llm-routing/docs/cloudshell-tutorial.md)

## Test Sample

### Authenticate your notebook environment (Colab only)
If you are running this notebook on Google Colab, run the following cell to authenticate your environment. This step is not required if you are using Vertex AI Workbench.

In [ ]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### Select a Provider

In [ ]:
PROVIDER = "anthropic"  # @param ["google", "anthropic" ]
if PROVIDER == "google":
    available_models = ["gemini-1.5-flash-001"]
    payload = {
      "contents":{
         "role":"user",
         "parts":[
            {
               "text":"Suggest name for a flower shop"
            }
         ]
      }
}
elif PROVIDER == "anthropic":
    available_models = ["claude-3-5-sonnet-v2@20241022"]
    payload = {
    "anthropic_version": "vertex-2023-10-16",
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Suggest name for a flower shop"
                }
            ]
        }
    ],
    "max_tokens": 256,
    "stream": False
}

### Select a Model

In [ ]:
import json
import ipywidgets as widgets
from IPython.display import display

modelDropdown = widgets.Dropdown(
    options=available_models,
    description="Model:",
    font_weight="bold"
)

payloadTextarea = widgets.Textarea(
    description="Payload:",
    font_weight="bold",
    width="100%",
    value=json.dumps(payload, indent=2)
)


def dropdown_eventhandler(change):
    global VALUE
    if change["type"] == "change" and change["name"] == "value":
        VALUE = change.new
        print("Selected:", change.new)


MODEL = modelDropdown.value
modelDropdown.observe(dropdown_eventhandler, names="value")
display(modelDropdown)

PAYLOAD = payloadTextarea.value
display(payloadTextarea)

#### Set Google Cloud project

To get started, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
APIGEE_HOST="[your-apigee-host-domain]" # @param {type:"string"}
APIKEY="[your-apikey]" # @param {type:"string"}

if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    raise ValueError("Please set your PROJECT_ID")
if not APIGEE_HOST or APIGEE_HOST == "[your-apigee-host-domain]":
    raise ValueError("Please set your APIGEE_HOST")
if not APIKEY or APIKEY == "[your-apikey]":
    raise ValueError("Please set your APIKEY")

#### Import required libraries

In [ ]:
import base64
import json

import httpx
import requests
from IPython.display import Image

### Text generation

#### Unary call

Sends a POST request to the specified API endpoint to get a response from the model using the provided payload.

In [ ]:
!curl -X POST -H "x-apikey: {APIKEY}" -H "Content-Type: application/json" https://{APIGEE_HOST}/v1/samples/llm-routing/providers/{PROVIDER}/models/{MODEL}:generateText -d '{PAYLOAD}'

To test different providers, just select another provider from the dropdown in the "Select a Provider" section above and execute each step to see different responses